Agents は実行するアクションとその順序を設定し、ツールを実行してその出力を取得できます

- zero-shot-react-description (ZeroShotAgent)
ReActフレームワークを使用して、ツールの説明のみに基づいて使用するツールを決定します。ツールはいくつでも提供できます。 このエージェントでは、ツールごとに説明を提供する必要があります。

-  react-docstore (ReActDocstoreAgent)
ReActフレームワークを使用してドキュメントストアと対話します。SearchとLookupの2つのツールを提供する必要があります。Searchはドキュメント、Lookupは最近見つかったドキュメント内の用語を検索します。
- self-ask-with-search (SelfAskWithSearchAgent)
Intermediate Answerという1つのツールを使用します。このツールは、質問に対する事実に基づく回答を検索できる必要があります。

In [1]:
!pip install langchain==0.0.152
!pip3 install rwkv tokenizer
!pip install google-search-results
!pip install wikipedia

In [2]:
# model
# !wget -P ./models https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-14B-v8-EngAndMore-20230408-ctx4096.pth

In [3]:
# !wget -P ./models https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192

In [4]:
# !wget -P ./tokenizer https://raw.githubusercontent.com/BlinkDL/ChatRWKV/main/20B_tokenizer.json

In [5]:
import gc
gc.collect()

227

In [6]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [7]:
%%time
# from langchain.llms import OpenAI

# # LLMの準備
# llm = OpenAI(temperature=0.9)

import os
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '0'
# os.environ["RWKV_CUDA_ON"] = '1'

# from rwkv.model import RWKV
# from rwkv.utils import PIPELINE, PIPELINE_ARGS

from langchain.llms import RWKV
with Timer(prefix=f'set model'):
    # LLMの準備
    llm = RWKV(
        model="./models/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192.pth", 
        # model="./models/RWKV-4-Raven-14B-v8-EngAndMore-20230408-ctx4096.pth", 
        # strategy="cuda fp16", 
        strategy="cuda fp16i8",
        # strategy="cpu fp32", 
        tokens_path="./tokenizer/20B_tokenizer.json",
        # temperature=1.0,
        temperature=0.25,
    )

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading ./models/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192.pth ...
Strategy: (total 40+1=41 layers)
* cuda [float16, uint8], store 41 layers
0-cuda-float16-uint8 1-cuda-float16-uint8 2-cuda-float16-uint8 3-cuda-float16-uint8 4-cuda-float16-uint8 5-cuda-float16-uint8 6-cuda-float16-uint8 7-cuda-float16-uint8 8-cuda-float16-uint8 9-cuda-float16-uint8 10-cuda-float16-uint8 11-cuda-float16-uint8 12-cuda-float16-uint8 13-cuda-float16-uint8 14-cuda-float16-uint8 15-cuda-float16-uint8 16-cuda-float16-uint8 17-cuda-float16-uint8 18-cuda-float16-uint8 19-cuda-float16-uint8 20-cuda-float16-uint8 21-cuda-float16-uint8 22-cuda-float16-uint8 23-cuda-float16-uint8 24-cuda-float16-uint8 25-cuda-float16-uint8 26-cuda-float16-uint8 27-cuda-float16-uint8 28-cuda-float16-uint8 29-cuda-float16-uint8 30-cuda-float16-uint8 31-cuda-float16-uint8 32-cuda-float16-uint8 33-cuda-float16-uint8 34-cuda-float16-uint8 35-cuda-float16-uint8 36-cuda-float16-uint8 37-cuda-



- [ReAct](https://python.langchain.com/en/latest/modules/agents/agents/examples/react.html)

In [8]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

In [9]:
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [10]:
with Timer(prefix=f'ReAct:'):
    # question = "指輪物語の作者J.R.Rトールキンの他の著作は?"
    question = "What other writings of J. R. R Tolkien, author of The Lord of the Rings?"
    react_res = react.run(question)
    
    # while react_res == 'Agent stopped due to iteration limit or time limit.':
    #     print('retry')
    #     react_res = react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search J. R. R Tolkien, find his other writings, then find if they are the same as The Lord of the Rings.
Action: Search[J. R. R Tolkien]
Observation: John Ronald Reuel Tolkien  (, ROOL TOL-keen; 3 January 1892 – 2 September 1973) was an English writer and philologist. He was the author of the high fantasy works The Hobbit and The Lord of the Rings.
From 1925 to 1945, Tolkien was the Rawlinson and Bosworth Professor of Anglo-Saxon and a Fellow of Pembroke College, both at the University of Oxford. He then moved within the same university to become the Merton Professor of English Language and Literature and Fellow of Merton College, and held these positions from 1945 until his retirement in 1959. Tolkien was a close friend of C. S. Lewis, a co-member of the informal literary discussion group The Inklings. He was appointed a Commander of the Order of the British Empire by Queen Elizabeth II on 28 March 1972.
After Tolkien's death

In [11]:
react_res

'The Hobbit'